In [2]:
import awswrangler  as wr
import pandas as pd
import pymysql
import datetime

In [27]:


# def lambda_handler(event, context):
    
#     sql_query = """
#     select title, 
#           case when navigation = 'contents_detail' then 'contents' else 'guide' end as is_contents, 
#           count(distinct uuid) as user_count, 
#           date_ymd
#     from
#     (SELECT date(date_ymd) as date_ymd,uuid,
#           object_id as title,
#           navigation
#     FROM "moyo_logs"."user_behavior_logs" 
#     where navigation in ('guide_detail', 'contents_detail') and category = 'pageview'
#           and date(date_ymd) = date_add('day',-1,date(current_timestamp at time zone 'asia/seoul')))
#     group by 1,2,4
#     """

#     df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

#     bucket = 'moyo-mart'
#     table_name = 'guide_visit_count_by_title'
    
#     res = wr.s3.to_parquet(
#         df=df,
#         path=f"s3://{bucket}/{table_name}/",
#         dataset=True,
#         database="moyo_mart",
#         table=table_name,
#         mode="overwrite_partitions",
#         partition_cols=['date_ymd']
#     )

In [40]:
sql = """
WITH raw1 AS (
  SELECT
    date_ymd,
    client_access_time,
    category,
    object_type,
    rand() AS rand,
    -- current_timestamp in KR time, add 9 hours
--     current_timestamp AT TIME ZONE 'UTC' + INTERVAL '9' HOUR AS current_timestamp_kr,
    -- Reformat client_access_time to the desired format
    CONCAT(substr(CAST( current_timestamp AT TIME ZONE 'UTC' + INTERVAL '9' HOUR  AS varchar), 1, 23), '000') AS reformatted_client_access_time
  FROM
    moyo_logs.user_behavior_logs
)
SELECT *
FROM
  raw1
ORDER BY
  rand
LIMIT 100;

"""

df = wr.athena.read_sql_query(sql=sql, database="moyo_order")

In [41]:
df.head()

,date_ymd,client_access_time,category,object_type,rand,reformatted_client_access_time
0,2022-12-14,2022-12-14 17:40:53,impression,plan,5.440454e-09,2023-05-09 08:03:55.953000
1,2022-10-13,2022-10-13 16:38:10,pageview,<NA>,7.604807e-09,2023-05-09 08:03:55.953000
2,2023-04-27,2023-04-27 15:17:10,pageexit,<NA>,1.342311e-08,2023-05-09 08:03:55.953000
3,2023-01-28,2023-01-28 13:45:20,pageview,plan,2.283086e-08,2023-05-09 08:03:55.953000
4,2022-12-30,2022-12-30 22:55:18,impression,plan,2.794881e-08,2023-05-09 08:03:55.953000


In [42]:
bucket = 'moyo-mart'
table_name = 'ringo_partition_test'


res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    partition_cols=['date_ymd']
)
# https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false


/Users/ringokwon/anaconda3/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:93: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


In [3]:
sql_query = """
    select 'c' as append, '2021-01-07' as date_ymd
    , CONCAT(substr(CAST( current_timestamp AT TIME ZONE 'UTC' + INTERVAL '9' HOUR  AS varchar), 1, 23), '000') AS reformatted_client_access_time
    union all 
     select 'c' as append, '2021-01-05' as date_ymd
    , CONCAT(substr(CAST( current_timestamp AT TIME ZONE 'UTC' + INTERVAL '9' HOUR  AS varchar), 1, 23), '000') AS reformatted_client_access_time
    """

df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

bucket = 'moyo-mart'
table_name = 'mart_test1'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions",
    partition_cols=['date_ymd']
    )

/Users/ringokwon/anaconda3/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:93: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


In [7]:
sql_query = """
    select 'd' as append
    , CONCAT(substr(CAST( current_timestamp AT TIME ZONE 'UTC' + INTERVAL '9' HOUR  AS varchar), 1, 23), '000') AS reformatted_client_access_time
    , 1 as additional_col
    ,'2021-01-03' as date_ymd
    """

df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

bucket = 'moyo-mart'
table_name = 'mart_test2'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions",
    partition_cols=['date_ymd']
    )

/Users/ringokwon/anaconda3/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:93: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):
